In [ ]:
import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader, TensorDataset, random_split

from transformers import BertTokenizer, BertForSequenceClassification
import pandas as pd
from sklearn.metrics import confusion_matrix

In [8]:
from transformers import BertForSequenceClassification
data = pd.read_csv('data.csv')

model_name = 'bert-base-uncased'

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch

# Check PyTorch version and GPU availability
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())

# Select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using Device:", device)

# If GPU is available, print GPU details
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

# Move model to GPU
model.to(device)

# Example tensor operation on GPU
x = torch.randn(3, 3).to(device)
print("Tensor on GPU:", x)

PyTorch Version: 2.6.0+cu126
CUDA Available: True
Using Device: cuda
GPU Name: NVIDIA GeForce RTX 4050 Laptop GPU
Tensor on GPU: tensor([[ 0.8252,  0.8965, -0.7408],
        [-0.9591, -0.2712, -0.1366],
        [ 1.8818,  0.6638, -0.3735]], device='cuda:0')


In [10]:
text_review = data['text_'].tolist()
label = data['label'].apply(lambda x: 1 if x == "OR" else 0).tolist()

In [ ]:
encoded_data = tokenizer(text_review, padding=True, truncation=True, return_tensors='pt')
encoded_data

{'input_ids': tensor([[  101,  2293,  2023,  ...,     0,     0,     0],
        [  101,  2293,  2009,  ...,     0,     0,     0],
        [  101,  2023, 10005,  ...,     0,     0,     0],
        ...,
        [  101,  2017,  2064,  ...,     0,     0,     0],
        [  101,  1045,  4669,  ...,     0,     0,     0],
        [  101,  1045,  2147,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [12]:
input_ids = encoded_data['input_ids']
attention_mask = encoded_data['attention_mask']
labels = torch.tensor(label)

In [13]:
dataset = TensorDataset(input_ids, attention_mask, labels)

In [14]:
#  Define dataset sizes
train_size = int(0.7 * len(dataset))  # 70% for training
valid_size = int(0.15 * len(dataset))  # 15% for validation
test_size = len(dataset) - train_size - valid_size  # 15% for testing

# Split dataset
train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=4, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=4, shuffle=False)

In [15]:
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

In [16]:
def evaluate_model(model, valid_dataloader):
    """
    Evaluates the model on the validation dataset and returns the average loss.
    
    :param model: The PyTorch model
    :param valid_dataloader: The validation data loader
    :return: Average validation loss
    """
    model.eval()
    total_val_loss = 0

    with torch.no_grad():
        for batch in valid_dataloader:
            inputs = {
                'input_ids': batch[0].to(model.device),
                'attention_mask': batch[1].to(model.device),
                'labels': batch[2].to(model.device)
            }

            output = model(**inputs)
            loss = output.loss
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(valid_dataloader)
    return avg_val_loss

In [17]:
import torch
import time
import numpy as np

def train_model(model, train_dataloader, valid_dataloader, optimizer, num_epochs=3, accumulation_steps=4, 
                pause_interval=120, pause_duration=80, patience=3):
    """
    Trains the model while implementing early stopping and periodic pauses.

    :param model: The PyTorch model
    :param train_dataloader: The training data loader
    :param valid_dataloader: The validation data loader (for early stopping)
    :param optimizer: The optimizer
    :param num_epochs: Number of epochs (default: 3)
    :param accumulation_steps: Number of steps before optimizer step (default: 4)
    :param pause_interval: Time before pausing execution (default: 120s)
    :param pause_duration: Time to sleep during pause (default: 60s)
    :param patience: Number of epochs to wait for validation loss improvement before stopping (default: 3)
    """

    best_val_loss = float('inf')
    no_improve_epochs = 0  # Count epochs without improvement

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        start_time = time.time()
        last_pause_time = start_time

        print(f"\n🚀 Epoch {epoch + 1}/{num_epochs} Starting...\n")

        for step, batch in enumerate(train_dataloader):
            inputs = {
                'input_ids': batch[0].to(model.device),
                'attention_mask': batch[1].to(model.device),
                'labels': batch[2].to(model.device)
            }

            output = model(**inputs)
            loss = output.loss / accumulation_steps  # Normalize loss for accumulation
            total_loss += loss.item()

            loss.backward()

            # Perform optimizer step only after 'accumulation_steps' batches
            if (step + 1) % accumulation_steps == 0:
                optimizer.step()
                optimizer.zero_grad()

            # Print progress every 500 batches
            if (step + 1) % 500 == 0 or (step + 1) == len(train_dataloader):
                print(f"  🔹 Step {step + 1}/{len(train_dataloader)}, Loss: {loss.item():.4f}")

        # Compute average loss for the epoch
        avg_loss = total_loss / len(train_dataloader)
        epoch_time = time.time() - start_time
        print(f"\n✅ Epoch {epoch + 1} Completed - Avg Loss: {avg_loss:.4f}, Time: {epoch_time:.2f} sec\n")

        # Validate the model if a validation set is provided
        if valid_dataloader:
            val_loss = evaluate_model(model, valid_dataloader)
            print(f"🔍 Validation Loss: {val_loss:.4f}\n")

            # Check for improvement
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                no_improve_epochs = 0  # Reset counter if improvement is seen
            else:
                no_improve_epochs += 1  # Increment counter if no improvement

            # Early stopping condition
            if no_improve_epochs >= patience:
                print(f"\n⏹️ Early Stopping Triggered! No improvement for {patience} epochs.\n")
                break

    print("\n🎉 Training Complete!\n")

In [18]:
train_model(model=model, 
            train_dataloader=train_dataloader, 
            valid_dataloader=valid_dataloader, 
            optimizer=optimizer, 
            num_epochs=20, 
            patience=3)



🚀 Epoch 1/20 Starting...

  🔹 Step 500/7076, Loss: 0.0113
  🔹 Step 1000/7076, Loss: 0.0036
  🔹 Step 1500/7076, Loss: 0.0111
  🔹 Step 2000/7076, Loss: 0.0147
  🔹 Step 2500/7076, Loss: 0.0814
  🔹 Step 3000/7076, Loss: 0.0160
  🔹 Step 3500/7076, Loss: 0.2029
  🔹 Step 4000/7076, Loss: 0.0048
  🔹 Step 4500/7076, Loss: 0.0047
  🔹 Step 5000/7076, Loss: 0.0023
  🔹 Step 5500/7076, Loss: 0.0004
  🔹 Step 6000/7076, Loss: 0.0003
  🔹 Step 6500/7076, Loss: 0.0031
  🔹 Step 7000/7076, Loss: 0.1154
  🔹 Step 7076/7076, Loss: 0.0003

✅ Epoch 1 Completed - Avg Loss: 0.0296, Time: 1633.31 sec

🔍 Validation Loss: 0.1550


🚀 Epoch 2/20 Starting...

  🔹 Step 500/7076, Loss: 0.0003
  🔹 Step 1000/7076, Loss: 0.0050
  🔹 Step 1500/7076, Loss: 0.0001
  🔹 Step 2000/7076, Loss: 0.0002
  🔹 Step 2500/7076, Loss: 0.0011
  🔹 Step 3000/7076, Loss: 0.0001
  🔹 Step 3500/7076, Loss: 0.0002
  🔹 Step 4000/7076, Loss: 0.0079
  🔹 Step 4500/7076, Loss: 0.0002
  🔹 Step 5000/7076, Loss: 0.0102
  🔹 Step 5500/7076, Loss: 0.0001
  🔹

In [19]:
import torch

model.eval()  # Set model to evaluation mode
all_predictions = []
all_labels = []

with torch.no_grad():
    total_correct = 0
    total_samples = 0
    batch_num = 0  # Track batch number

    print("Starting evaluation...\n")

    for batch in test_dataloader:
        batch_num += 1
        inputs = {
            'input_ids': batch[0].to(model.device),
            'attention_mask': batch[1].to(model.device),
            'labels': batch[2].to(model.device)
        }

        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

        # Compute batch accuracy
        correct = torch.sum(predictions == inputs['labels']).item()
        batch_size = inputs['labels'].size(0)
        batch_accuracy = correct / batch_size

        total_correct += correct
        total_samples += batch_size
        overall_accuracy = total_correct / total_samples

        # Store predictions and labels
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(inputs['labels'].cpu().numpy())

        # Print verbose details for each batch
        print(f"Batch {batch_num}:")
        print(f"  - Batch Size: {batch_size}")
        print(f"  - Correct Predictions: {correct}/{batch_size}")
        print(f"  - Batch Accuracy: {batch_accuracy:.4f}")
        print(f"  - Running Accuracy: {overall_accuracy:.4f}\n")

    print("Evaluation completed.")
    print(f"Final Accuracy: {overall_accuracy:.4f}")


Starting evaluation...

Batch 1:
  - Batch Size: 4
  - Correct Predictions: 4/4
  - Batch Accuracy: 1.0000
  - Running Accuracy: 1.0000

Batch 2:
  - Batch Size: 4
  - Correct Predictions: 4/4
  - Batch Accuracy: 1.0000
  - Running Accuracy: 1.0000

Batch 3:
  - Batch Size: 4
  - Correct Predictions: 4/4
  - Batch Accuracy: 1.0000
  - Running Accuracy: 1.0000

Batch 4:
  - Batch Size: 4
  - Correct Predictions: 4/4
  - Batch Accuracy: 1.0000
  - Running Accuracy: 1.0000

Batch 5:
  - Batch Size: 4
  - Correct Predictions: 4/4
  - Batch Accuracy: 1.0000
  - Running Accuracy: 1.0000

Batch 6:
  - Batch Size: 4
  - Correct Predictions: 4/4
  - Batch Accuracy: 1.0000
  - Running Accuracy: 1.0000

Batch 7:
  - Batch Size: 4
  - Correct Predictions: 4/4
  - Batch Accuracy: 1.0000
  - Running Accuracy: 1.0000

Batch 8:
  - Batch Size: 4
  - Correct Predictions: 4/4
  - Batch Accuracy: 1.0000
  - Running Accuracy: 1.0000

Batch 9:
  - Batch Size: 4
  - Correct Predictions: 3/4
  - Batch Accura

In [20]:
cm = confusion_matrix(all_labels, all_predictions)
print("Confusion Matrix:")
print(cm)

Confusion Matrix:
[[2990   27]
 [ 191 2858]]


In [21]:
torch.save(model.state_dict(), "nlp_model.pth")

In [1]:
import torch

# Set device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
from transformers import BertForSequenceClassification, BertTokenizer

# Define model path
model_path = "nlp_model.pth"  # Change this to your model's file path

# Load tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# Load model
model = BertForSequenceClassification.from_pretrained(model_name)
model.load_state_dict(torch.load(model_path, map_location=device))  # Load model weights
model.to(device)  # Move model to GPU
model.eval()  # Set model to evaluation mode

print("Model loaded successfully on", device)

c:\Users\mradu\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully on cuda


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer

# Define model path
model_path = "nlp_model.pth"

# Load tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)

# Load model
model = BertForSequenceClassification.from_pretrained(model_name)
model.load_state_dict(torch.load(model_path, map_location=device))  # Load model weights
model.to(device)  # Move model to GPU
model.eval()  # Set model to evaluation mode

print("Model loaded successfully on", device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully on cuda


In [ ]:
def predict(text):
    """
    Predicts sentiment using the loaded model.
    
    :param text: Input sentence (string)
    :return: Model prediction
    """
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        prediction = torch.argmax(logits, dim=1).item()

    return "Positive" if prediction == 1 else "Negative"

# Example usage
sample_text = "These are just perfect, exactly what I was looking for."
print(predict(sample_text))

Positive


In [6]:
def batch_predict(sentences):
    """
    Predicts sentiment for a list of sentences using the model.
    
    :param sentences: List of input sentences (strings)
    :return: List of predictions
    """
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1).tolist()

    return ["Positive" if pred == 1 else "Negative" for pred in predictions]

# Example usage
sample_texts = ["Very nice set. Good quality. We have had the set for two months now and have not been", "Terrible experience.", "Fantastic product!", "Not worth it."]
print(batch_predict(sample_texts))


['Negative', 'Positive', 'Positive', 'Positive']
